In [7]:
import gc
import numpy as np
import pandas as pd
from keras import backend as K
from keras.callbacks import CSVLogger, ModelCheckpoint, Callback
from sklearn.model_selection import ParameterGrid
import datetime
import os
import numpy as np
import h5py
import pickle


from models import MDAD_model, single_MLP_model, single_linear_model
from experiment_helpers import load_final_PCA_data, save_MTL_predictions

from configs import * 

os.environ["CUDA_VISIBLE_DEVICES"]="2"
K.tensorflow_backend._get_available_gpus()

ModuleNotFoundError: No module named 'configs'

In [5]:
# hyperparams = {"epochs": [200], \
#                "nonlinearity": ["relu"], \
#                "hidden_sizes_shared": [[500,100]], \
#                "hidden_sizes_separate": [[50,10]],\
#                "dropout":  [.1],\
#                "k_reg": [.00001,.001],\
#                "learning_rate": [.0001,.001],\
#                "loss_weights":  [[1, 1]],\
#                "grad_clip_norm": [.01,.1],\
#                "batch_size": [20]}
# SPECIFIC_FOLDER = "origGE"
# dataset = "ACT_MSBBRNA_ROSMAP_PCA"



baselines_final_selected_model = pickle.load(open("../../AD_Project/analyses/MTL_variable_tasks/6vars-continuous/%s/final_models_chosen/MLP_baselines/final.p"%SPECIFIC_FOLDER, "rb" ) )
path_to_all = "../../md-ad_public_repo_data/Modeling/final_model/%s/"%SPECIFIC_FOLDER
phenotypes = ["ABETA_IHC", "TAU_IHC","PLAQUES", "TANGLES","BRAAK", "CERAD"]

In [3]:
path_to_results = path_to_all + "results/MLP_baselines/"
path_to_models = path_to_all + "models/MLP_baselines/"

hy_dict_list = list(ParameterGrid(hyperparams))

for hy_iteration in range(len(hy_dict_list)):

    hy_dict = hy_dict_list[hy_iteration]

    title = "%d_%s_%s_%s_%f_%f_%f_%s_%f_%d"%(hy_dict["epochs"], hy_dict["nonlinearity"], 
                        str(hy_dict["hidden_sizes_shared"]), str(hy_dict["hidden_sizes_separate"]),
                        hy_dict["dropout"], hy_dict["k_reg"], hy_dict["learning_rate"], 
                        str(hy_dict["loss_weights"]),  hy_dict["grad_clip_norm"], hy_dict["batch_size"])    
    print(title)


200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.000100_[1, 1]_0.010000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.001000_[1, 1]_0.010000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.000100_[1, 1]_0.010000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.001000_[1, 1]_0.010000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.000100_[1, 1]_0.100000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.001000_[1, 1]_0.100000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.000100_[1, 1]_0.100000_20
200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.001000_[1, 1]_0.100000_20


In [4]:
for rep in range(100):
  
    X,y = load_final_PCA_data(SPECIFIC_FOLDER)

    print(X.shape)

    for hy_iteration in range(len(hy_dict_list)):

        print("HYPERPARAMETER ITERATION: %d"%hy_iteration)

        hy_dict = hy_dict_list[hy_iteration]

        title = "%d_%s_%s_%s_%f_%f_%f_%s_%f_%d"%(hy_dict["epochs"], hy_dict["nonlinearity"], 
                        str(hy_dict["hidden_sizes_shared"]), str(hy_dict["hidden_sizes_separate"]),
                        hy_dict["dropout"], hy_dict["k_reg"], hy_dict["learning_rate"], 
                        str(hy_dict["loss_weights"]),  hy_dict["grad_clip_norm"], hy_dict["batch_size"])
        print(title)
            
        for phenotype in  phenotypes:
            print(phenotype)
            if title == baselines_final_selected_model[phenotype]:

                model = single_MLP_model(X, hy_dict) 

                modelpath = path_to_models + dataset + "/" + phenotype + "/" +title +  "/" + str(rep) + "/"
                res_dest = path_to_results +  "/" + dataset + "/" + phenotype +"/" + title + "/"+ str(rep) + "/"
                for path in [modelpath, res_dest]:
                    if not os.path.isdir(path):
                        os.makedirs(path)

                csv_logger = CSVLogger(res_dest+'results.log')

                History = model.fit(x={'main_input': X},y={'out': y[phenotype]}, 
                                    verbose=0,epochs=hy_dict["epochs"], batch_size=hy_dict["batch_size"], callbacks=[csv_logger,\
                                    ModelCheckpoint(modelpath+"{epoch:02d}.hdf5", monitor='val_loss', verbose=1, \
                                    save_best_only=False, save_weights_only=False, mode='auto', period=200)])

                K.clear_session()
                gc.collect()   

(1758, 500)
HYPERPARAMETER ITERATION: 0
200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.000100_[1, 1]_0.010000_20
ABETA_IHC
TAU_IHC
PLAQUES
TANGLES
BRAAK
CERAD
HYPERPARAMETER ITERATION: 1
200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.001000_[1, 1]_0.010000_20
ABETA_IHC


/home/nbbwang/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


InternalError: WhereOp: Could not launch cub::DeviceReduce::Sum to calculate temp_storage_bytes, status: out of memory
	 [[Node: loss/out_loss/boolean_mask/Where = Where[_device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/out_loss/boolean_mask/Reshape_1)]]
	 [[Node: loss/add_3/_139 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1162_loss/add_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'loss/out_loss/boolean_mask/Where', defined at:
  File "/home/nbbwang/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/nbbwang/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nbbwang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_i